In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join(r'D:\gitClones\nteract_models\optimize\scripts'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from scipy.optimize import minimize
from scripts.util import q_ge, spillway_powerhouse_flows, tailwater, powerhouse_entrainment
import pandas as pd

In [ ]:
data = {}
maxiter = 3
x0 = [1,.5,150]
min_fb_tdg = float('-inf')

In [ ]:
#put data in dataframe
data = pd.DataFrame(data['train'])
data['index'] = pd.to_datetime(data['index'])
data.set_index('index', inplace = True)
#separate only data with spill.  A separate process will be optimized with no spill conditions
spill = data[(data['q_s']>0) & (data['tdg_f']> min_fb_tdg)]

#create a new dataset if used for a bootstrap process
if sample:
    spill = spill.sample(frac = 1.0)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y, y_hat):
    return sqrt(mean_squared_error(y,y_hat))

In [ ]:
#See scripts/utils.py for explanation of spillway_powerhouse_flows, tailwater, and powerhouse_entrainment or reference

#Reference:
#Stewart, Kevin M., Adam Witt, and Boualem Hadjerioua. 
#"Total dissolved gas prediction and optimization in RIVERWARE." 
#Prepared for US Department of Energy Wind and Water Program by 
#Oakridge National Laboratory, Oak Ridge, TN (2015).
#https://info.ornl.gov/sites/publications/files/Pub59285.pdf


#The objective function is what the optimization will try to minimize with x, the b1, b2, and b3 coefficients.
def objective(x):
    b1,b2,b3 = x
    A = spill.apply(lambda r:spillway_powerhouse_flows(r['q_s'],b1,b3,r['q_p']), axis = 1)
    B = spill.apply(lambda r:tailwater(r['h_t'],r['temp_water'], r['p_atm']), axis = 1)
    C = spill.apply(lambda r:powerhouse_entrainment(r['q_p'],b1,r['q_s'], b3), axis = 1)
    tdg_f = spill['tdg_f']
    y_hat = 100*A*B*b2+tdg_f*C
    y = spill['tdg_tw']
    return rmse(y, y_hat)

#constraints for optimization model.  See reference for more information

def constraint_b2(x):
    b1,b2,b3=x
    return 0-b2

def constraint_q_ge(x):
    ##This may be wrong
    b1,b2,b3=x
    power_flow = spill.apply(lambda x:q_ge(x['q_p'],x['q_s'],b1,b3), axis = 1)
    
    return 0 - power_flow.sum()

constraints = [
                {'type':'ineq', 'fun':constraint_b2},
                {'type':'ineq', 'fun':constraint_q_ge},
              ]


In [ ]:
#optimization process
sol = minimize(objective, x0, method = 'SLSQP', constraints = constraints, options={'disp':True, 'maxiter':maxiter})

In [ ]:
#coefficients b1, b2, b3
x=list(sol.x)


In [ ]:

import papermill as pm

pm.record('x', x)

